In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle
import matplotlib.pyplot as plt

In [2]:
from process import gen_second_data

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [4]:
df = pd.read_csv(FOLDER+'train.csv')
df_structures = pd.read_csv(FOLDER+'structures.csv')

In [5]:
df_idx = df.set_index('molecule_name')
df_structures_idx = df_structures.set_index('molecule_name')
mols_train = df['molecule_name'].unique()

In [6]:
pred_train = '20190721_predict_train.pickle'
pred_val = '20190721_predict_val.pickle'
first_submission_name = '20190721_submission.csv'
temp_no = 0
temp_pickle = '20190721_features_{}.pickle'.format(temp_no)
temp_pickle_val = '20190721_features_val_{}.pickle'.format(temp_no)
temp_pickle_sub = '20190721_features_sub_{}.pickle'.format(temp_no)

In [7]:
df_train = pd.read_pickle(OUTPUT + pred_train)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]
df_val = pd.read_pickle(OUTPUT + pred_val)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]

mols_train = df_train['molecule_name'].unique()
mols_val = df_val['molecule_name'].unique()

df = pd.concat([df_train, df_val], axis=0).sort_values('id')
df_idx = df.set_index('molecule_name')

In [8]:
bonds = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']

In [ ]:
div = len(mols_train) // 8

df_train_features = pd.DataFrame()
start0 = time.time()

for m in mols_train[div*temp_no:div*(temp_no+1)]:
# for m in mols_train[:100]:
    for b in bonds:
        df_feature = gen_second_data(df_idx, df_structures_idx, m, target_bond=b, target_col='predict')
        df_train_features = pd.concat([df_train_features, df_feature], axis=0)

df_train_features.to_pickle(OUTPUT + temp_pickle)
elapsed_time = time.time() - start0
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
pd.merge(df, df_train_features, on=['id'], how='inner')['type'].value_counts()

In [ ]:
div = len(mols_val) // 8

df_val_features = pd.DataFrame()
start0 = time.time()

for m in mols_val[div*temp_no:div*(temp_no+1)]:
# for m in mols_val[:100]:
    for b in bonds:
        df_feature = gen_second_data(df_idx, df_structures_idx, m, target_bond=b, target_col='predict')
        df_val_features = pd.concat([df_val_features, df_feature], axis=0)
    
df_val_features.to_pickle(OUTPUT + temp_pickle_val)
elapsed_time = time.time() - start0
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_test_pred = pd.read_csv(OUTPUT + first_submission_name).rename(columns={'scalar_coupling_constant':'predict'})
# df_test_merge = pd.merge(df_test, df_test_pred, on=['id'], how='inner')

# df_test_idx = df_test_merge.set_index('molecule_name')
# mols_test = df_test['molecule_name'].unique()

In [ ]:
# div = len(mols_test) // 8

# df_test_features = pd.DataFrame()
# start0 = time.time()

# for m in mols_test[div*temp_no:div*(temp_no+1)]:
# # for m in mols_test[:100]:
#     for b in bonds:
#         df_feature = gen_second_data(df_test_idx, df_structures_idx, m,  target_bond=b, target_col='predict')
#         df_test_features = pd.concat([df_test_features, df_feature], axis=0)
    
# df_test_features.to_pickle(OUTPUT + temp_pickle_sub)
# elapsed_time = time.time() - start0
# print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")